In [1]:
# -*- coding: utf-8 -*-
"""ENPM661_project3.ipynb

Automatically generated by Colab.
"""

from numpy import cos, sin, deg2rad, round, sqrt
import time
import heapq
import cv2
import numpy as np

# Define canvas dimensions (height and width)
canvas_height, canvas_width = 300, 540
wheel_radius = 3.6
wheel_distance = int(235 / 100) * 10  # Distance between wheels in mm
robot_radius = int(33.5 / 2)  # Radius of the robot in mm

# Define the Node class to store information about each node in the search space
class Node:
    """Class to store the node information."""
    def __init__(self, coords, cost, parent=None, heuristic=0):
        # Coordinates of the node (x, y, theta)
        self.coords = coords  # Node's position (x, y, theta)
        self.x = coords[0]  # x-coordinate
        self.y = coords[1]  # y-coordinate
        self.orientation = coords[2]  # Orientation (theta)
        self.cost = cost  # Cost to reach this node from the start
        self.parent = parent  # Parent node for backtracking
        self.heuristic = heuristic  # Heuristic cost to the goal

    def __lt__(self, other):
        """Comparison operator for priority queue."""
        # Compare nodes based on their total cost (cost + heuristic)
        return self.cost + self.heuristic < other.cost + other.heuristic


# Define obstacle-checking functions for different shapes (letters and numbers)
def is_in_obstacle_rect1(x, y, x0=100, y0=200, width=10, height=200, thickness=3):
    """Checks if a point (x, y) is inside the number '1'."""
    # Vertical bar of "1"
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= canvas_height:
        return True
    return False


def is_in_obstacle_rect2(x, y, x0=210, y0=300, width=10, height=200, thickness=3):
    """Checks if a point (x, y) is inside the number '1'."""
    # Vertical bar of "1"
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= height:
        return True
    return False


def is_in_obstacle_rect3(x, y, x0=320, y0=300, width=10, height=100, thickness=3):
    """Checks if a point (x, y) is inside the number '1'."""
    # Vertical bar of "1"
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= height:
        return True
    return False


def is_in_obstacle_rect4(x, y, x0=320, y0=100, width=10, height=100, thickness=3):
    """Checks if a point (x, y) is inside the number '1'."""
    # Vertical bar of "1"
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= canvas_height:
        return True
    return False


def is_in_obstacle_rect5(x, y, x0=430, y0=200, width=10, height=200, thickness=3):
    """Checks if a point (x, y) is inside the number '1'."""
    # Vertical bar of "1"
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= canvas_height:
        return True
    return False


def generate_map():
    """Creates the map with obstacles and clearance zones."""
    # Initialize an empty grid (canvas) to represent the environment
    grid = np.zeros((canvas_height, canvas_width), dtype=np.uint8)
    clearance_mask = np.zeros((canvas_height, canvas_width), dtype=np.uint8)

    # Calculate the offset for clearance and robot radius
    offset = clearance + robot_radius

    # Traverse through each pixel in the canvas
    for i in range(canvas_height):
        for j in range(canvas_width):
            # Check if the pixel is inside any obstacle
            if (is_in_obstacle_rect1(j, i) or
                is_in_obstacle_rect2(j, i) or
                is_in_obstacle_rect3(j, i) or
                is_in_obstacle_rect4(j, i) or
                is_in_obstacle_rect5(j, i)):
                grid[i][j] = 1  # Mark as obstacle

    # Dilate the grid to account for clearance
    kernel = np.ones((int(offset * 2 + 1), int(offset * 2 + 1)), np.uint8)
    clearance_mask = cv2.dilate(grid, kernel, iterations=1)

    # Ensure obstacle pixels remain distinguishable inside the clearance mask
    clearance_mask[np.where(grid == 1)] = 2  # Assign distinct value to obstacles
    # clearance_mask[0:offset, :] = 1  # Top boundary
    # clearance_mask[canvas_height-offset:canvas_height, :] = 1  # Bottom boundary
    # clearance_mask[:, 0:offset] = 1  # Left boundary
    # clearance_mask[:, canvas_width-offset:canvas_width] = 1  # Right boundary
    grid = clearance_mask
    return grid, clearance_mask


def is_valid_point(x, y, grid):
    """Checks whether the given coordinates are valid (not on an obstacle)."""
    # Return False if the point is on an obstacle
    if grid[y][x] == 1:
        return False
    return True


def create_animation_grid():
    """Generates visualization grid with color-coded obstacles and clearance zones."""
    # Initialize canvas with white background
    image = np.full((canvas_height, canvas_width, 3), 255, dtype=np.uint8)
    offset = clearance + robot_radius

    # Color definitions (BGR format)
    colors = {
        'clearance': (0, 255, 255),   # Yellow
        'obstacle': (255, 0, 0)       # Blue
    }

    # Create obstacle mask
    obstacle_mask = np.zeros((canvas_height, canvas_width), dtype=bool)
    obstacle_funcs = [
        is_in_obstacle_rect1, is_in_obstacle_rect2,
        is_in_obstacle_rect3, is_in_obstacle_rect4,
        is_in_obstacle_rect5
    ]

    # Vectorized obstacle check
    for i in range(canvas_height):
        for j in range(canvas_width):
            obstacle_mask[i, j] = any(f(j, i) for f in obstacle_funcs)

    # Create clearance zones using dilation
    kernel = np.ones((offset*2 + 1, offset*2 + 1), np.uint8)
    clearance_mask = cv2.dilate(obstacle_mask.astype(np.uint8), kernel)

    # Apply colors in priority order
    # clearance_mask[:offset, :] = 1
    # clearance_mask[-offset:, :] = 1
    # clearance_mask[:, :offset] = 1
    # clearance_mask[:, -offset:] = 1
    image[clearance_mask.astype(bool) & ~obstacle_mask] = colors['clearance']
    image[obstacle_mask] = colors['obstacle']
    return image


def round_value(number):
    return round(number * 2) / 2


def is_valid_child(child):
    """Checks whether the child node is valid (not outside the canvas or on an obstacle)."""
    x, y, _ = child[0]
    # Check if the child node is within the canvas boundaries
    if 0 <= x <= canvas_width - 1 and 0 <= y <= canvas_height - 1:
        # Check if the child node is not on an obstacle
        if canvas[int(round(y))][int(round(x))] == 0:
            return True
    return False


def differential_drive_action(x, y, theta, UL, UR, dt=0.05, steps=20):
    """
    Simulates motion of a differential drive robot using wheel RPMs.
    UL: Left wheel RPM
    UR: Right wheel RPM
    """
    t = 0
    theta = deg2rad(theta)
    Xn, Yn, Thetan = x, y, theta
    path = []
    for _ in range(steps):
        t += dt
        # Convert RPM to rad/s
        ul_rad = (UL * 2 * np.pi) / 60
        ur_rad = (UR * 2 * np.pi) / 60
        # Compute linear and angular velocities
        dx = 0.5 * wheel_radius * (ul_rad + ur_rad) * np.cos(Thetan) * dt
        dy = 0.5 * wheel_radius * (ul_rad + ur_rad) * np.sin(Thetan) * dt
        dtheta = (wheel_radius / wheel_distance) * (ur_rad - ul_rad) * dt
        Xn += dx
        Yn += dy
        Thetan += dtheta
        path.append((Xn, Yn))
    return [Xn, Yn, np.rad2deg(Thetan)], path


def generate_successors(node):
    """Generates all valid successors based on differential drive actions."""
    x, y, theta = node.coords
    cost = node.cost
    successors = []
    rpm_pairs = [
        (0, RPM1_val), (RPM1_val, 0), (RPM1_val, RPM1_val),
        (0, RPM2_val), (RPM2_val, 0), (RPM2_val, RPM2_val),
        (RPM1_val, RPM2_val), (RPM2_val, RPM1_val)
    ]
    for ul, ur in rpm_pairs:
        new_coords, path = differential_drive_action(x, y, theta, ul, ur)
        xn, yn, thetan = new_coords
        if 0 <= xn < canvas_width and 0 <= yn < canvas_height:
            # Clamp indices to be within valid bounds
            x_idx = min(int(round(xn)), canvas_width - 1)
            y_idx = min(int(round(yn)), canvas_height - 1)
            if canvas[y_idx][x_idx] == 0:
                rounded_coords = [round_value(xn), round_value(yn), thetan % 360]
                new_cost = cost + np.linalg.norm([xn - x, yn - y])
                successors.append([rounded_coords, new_cost])
    return successors


current_pose = [0.0, 0.0, 0.0]  # x (mm), y (mm), theta (degrees)


def update_robot_pose(ul, ur, dt=0.1):
    """
    Updates the robot's pose based on the left (ul) and right (ur) wheel RPMs.
    Args:
        ul (float): Rotational velocity (RPM) of the left wheel.
        ur (float): Rotational velocity (RPM) of the right wheel.
        dt (float): Time step duration in seconds. Default is 0.1 sec.
    Returns:
        tuple: New pose of the robot as (x, y, theta).
               x and y are in mm, theta in degrees.
    """
    global current_pose
    x, y, theta = current_pose  # Retrieve the current pose
    # Convert RPM to rad/s
    ul_rad = (ul * 2 * np.pi) / 60.0
    ur_rad = (ur * 2 * np.pi) / 60.0
    # Compute the linear velocity (v) and angular velocity (omega)
    v = (wheel_radius / 2.0) * (ul_rad + ur_rad)  # in mm/s
    omega = (wheel_radius / wheel_distance) * (ur_rad - ul_rad)  # in rad/s
    # Convert theta from degrees to radians for computation
    theta_rad = deg2rad(theta)
    # Update pose using forward Euler integration
    new_x = x + v * np.cos(theta_rad) * dt
    new_y = y + v * np.sin(theta_rad) * dt
    new_theta = theta + np.rad2deg(omega * dt)  # convert change in angle back to degrees
    # Store the new pose globally
    current_pose = [new_x, new_y, new_theta]
    return (new_x, new_y, new_theta)


def check_visited(x, y, theta):
    """
    Checks duplicate nodes by discretizing x, y, and theta,
    ensuring indices are within bounds of visited_mat.
    """
    # Convert continuous coordinates into discrete indices:
    xn = int(round_value(x) / visited_threshold)
    yn = int(round_value(y) / visited_threshold)
    # Clamp the x and y indices using the visited_mat dimensions
    xn = min(max(xn, 0), visited_mat.shape[1] - 1)  # shape[1] is width dimension
    yn = min(max(yn, 0), visited_mat.shape[0] - 1)  # shape[0] is height dimension
    # Normalize theta to be between 0 and 360, then discretize into the number of bins
    theta_mod = theta % 360
    num_bins = visited_mat.shape[2]  # For your code, num_bins is 12
    thetan = int(theta_mod / (360 / num_bins))
    thetan = min(max(thetan, 0), num_bins - 1)  # Clamp the orientation index as well
    # Now check and mark in the visited matrix
    if visited_mat[yn][xn][thetan] == 0:
        visited_mat[yn][xn][thetan] = 1
        return True
    return False


def astar_search(start_node, goal_node):
    """Performs A* search to find the shortest path."""
    # Create the animation canvas
    animation_grid = create_animation_grid()
    animation_frames = []
    # Initialize data structures for A* algorithm
    node_graph = {}
    open_list = {}
    closed_list = {}
    queue = []
    # Add the start node to the open list and priority queue
    open_list[str([start_node.x, start_node.y])] = start_node
    heapq.heappush(queue, [start_node.cost, start_node])
    i = 0  # Counter for animation frames
    while len(queue) != 0:
        # Fetch the node with the lowest cost from the priority queue
        fetched_ele = heapq.heappop(queue)
        current_node = fetched_ele[1]
        # Add the current node to the node graph
        node_graph[str([current_node.x, current_node.y])] = current_node
        # Draw the current node on the animation canvas
        cv2.circle(animation_grid, (int(current_node.x), int(current_node.y)), 1, (0, 0, 255), -1)
        # Check if the goal has been reached
        if sqrt((current_node.x - goal_node.x) ** 2 + (current_node.y - goal_node.y) ** 2) < goal_threshold:
            goal_node.parent = current_node.parent
            goal_node.cost = current_node.cost
            print("Found the goal node")
            break
        # Add the current node to the closed list
        if str([current_node.x, current_node.y]) in closed_list:
            continue
        else:
            closed_list[str([current_node.x, current_node.y])] = current_node
        # Remove the current node from the open list
        del open_list[str([current_node.x, current_node.y])]
        # Generate children nodes for the current node
        child_list = generate_successors(current_node)
        for child in child_list:
            child_x, child_y, child_theta = child[0]
            child_cost = child[1]
            # Draw the child node on the animation canvas
            cv2.circle(animation_grid, (int(child_x), int(child_y)), 1, (0, 255, 0), -1)
            # Save animation frames periodically
            if i % 300 == 0:
                animation_frames.append(animation_grid.copy())
            # Skip if the child node is already in the closed list
            if str([child_x, child_y]) in closed_list:
                continue
            # Calculate the heuristic cost to the goal
            child_heuristic = sqrt((goal_node.x - child_x) ** 2 + (goal_node.y - child_y) ** 2)
            # Create a new child node
            child_node = Node([child_x, child_y, child_theta], child_cost, current_node, child_heuristic)
            # Check if the child node has already been visited
            if check_visited(child_x, child_y, child_theta):
                if str([child_x, child_y]) in open_list:
                    # Update the cost if a cheaper path is found
                    if child_node.cost < open_list[str([child_x, child_y])].cost:
                        open_list[str([child_x, child_y])].cost = child_cost
                        open_list[str([child_x, child_y])].parent = current_node
                else:
                    # Add the child node to the open list and priority queue
                    open_list[str([child_x, child_y])] = child_node
                    heapq.heappush(queue, [(child_cost + child_heuristic), child_node])
        i += 1  # Increment the frame counter
    return node_graph, animation_grid, animation_frames


def backtrack_path(node_graph, goal_node):
    """Backtracks to find the path from start to goal, saving (x, y, theta) for each node."""
    path = []
    # Save the goal node with its orientation (rounded to int if desired)
    path.append([int(goal_node.x), int(goal_node.y), int(goal_node.orientation)])
    parent = list(node_graph.items())[-1][1].parent
    while parent:
        path.append([int(parent.x), int(parent.y), int(parent.orientation)])
        parent = parent.parent
    return path


def save_video(frames):
    """Saves the exploration animation as a video."""
    print("----------------------------")
    print("Generating the video file.")
    video = cv2.VideoWriter('path_planning.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 100, (540, 300), True)
    for frame in frames:
        video.write(frame)
        # cv2.waitKey(1)
    # cv2.destroyAllWindows()
    video.release()
    print("Video file generated successfully.")
    print("-------------------------------")


import matplotlib.pyplot as plt


def visualize_final_path(image, path):
    """
    Visualizes the final generated path on the canvas.
    This function overlays the path on the given image by marking the start (green),
    the goal (red), and drawing a line (magenta) connecting all the points in the path.
    Args:
        image (numpy.ndarray): The canvas image (e.g., generated by create_animation_grid()).
        path (list): A list of points representing the final path, where each point is [x, y].
    Returns:
        numpy.ndarray: The image with the overlaid path.
    """
    # Make a copy of the image so as not to modify the original
    vis = image.copy()
    # If path is empty, display the original image
    if not path:
        print("No path to display.")
        plt.figure(figsize=(8, 6))
        vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
        plt.imshow(vis_rgb)
        plt.title("Canvas (No Path Found)")
        plt.axis("off")
        plt.show()
        return vis
    # Mark start and goal (assuming the first element is the goal and last is start based on backtracking order)
    start_point = tuple(path[-1])
    goal_point = tuple(path[0])
    cv2.circle(vis, start_point[:2], 4, (0, 255, 0), -1)  # green for start
    cv2.circle(vis, goal_point[:2], 4, (0, 0, 255), -1)   # red for goal
    # Draw lines between consecutive points along the path
    for i in range(1, len(path)):
        pt1 = tuple(path[i - 1])
        pt2 = tuple(path[i])
        cv2.line(vis, pt1[:2], pt2[:2], (255, 0, 255), 2)  # magenta line
    # Convert image from BGR (OpenCV default) to RGB for matplotlib display
    vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(vis_rgb)
    plt.title("Final Generated Path on Canvas")
    plt.axis("off")
    plt.show()
    return vis


from google.colab import files


def save_waypoints(path, filename="final_path.txt"):
    """
    Saves the final path waypoints to a text file.
    Each waypoint (x, y, theta) is saved on a new line.
    Args:
        path (list): A list of waypoints; each waypoint is a list or tuple [x, y, theta].
        filename (str): The name of the file to save. Defaults to "final_path.txt".
    """
    with open(filename, "w") as f:
        for point in path:
            # Write each waypoint as comma separated values
            f.write("{}, {}, {}\n".format(point[0], point[1], point[2]))
    # Trigger file download in Colab
    files.download(filename)


def main():
    """Main function to run the program."""
    global canvas, canvas_height, canvas_width
    global step_size, clearance, robot_radius
    global goal_threshold, visited_mat, visited_threshold
    # Initialize global parameters
    canvas_height, canvas_width = 300, 540
    visited_threshold = 0.5 * 2
    visited_mat = np.zeros((500, 1200, 8), dtype=np.uint8)
    goal_threshold = 1.5
    loop = True
    while loop:
        start_x, start_y, start_theta = map(int, input("\nEnter the start point (x y theta): ").split())
        goal_x, goal_y = map(int, input("\nEnter the goal point (x y): ").split())
        RPM1, RPM2 = map(int, input("\nEnter the two wheel RPMs (RPM1 RPM2): ").split())
        clearance = int(input("\nEnter the clearance: "))
        global RPM1_val, RPM2_val
        RPM1_val, RPM2_val = RPM1, RPM2
        if(start_x > canvas_width or start_y > canvas_height or goal_x > canvas_width or goal_y > canvas_height):
            print("\nThe start point or goal point is out of bounds")
            continue
        # Create the canvas with obstacles
        canvas, clearance_mask = generate_map()
        if is_valid_point(start_x, start_y, canvas):
            if is_valid_point(goal_x, goal_y, canvas):
                print("\nStart Point = [", start_x, ", ", start_y, ", ", start_theta, "]")
                print("Goal Point = [", goal_x, ", ", goal_y, "]")
                print("Clearance = ", clearance)
                print("Robot Radius = ", robot_radius)
                start_y = canvas_height - start_y - 1
                goal_y = canvas_height - goal_y - 1
                # Define the start and goal nodes
                start_node = Node([start_x, start_y, start_theta], 0)
                goal_node = Node([goal_x, goal_y, 0], 0)
                print("\nFinding the goal node...")
                start_time = time.time()
                node_graph, animation_grid, animation_frames = astar_search(start_node, goal_node)
                final_path = backtrack_path(node_graph, goal_node)
                end_time = time.time()
                print("\nThe output was processed in ", end_time - start_time, " seconds.")
                print("------------------------------------------")
                for point in final_path[::-1]:
                    center = (point[0], point[1])  # Only x and y
                    cv2.circle(animation_grid, center, 1, (255, 0, 0), -1)
                    animation_frames.append(animation_grid.copy())
                animation_grid = create_animation_grid()
                final_image = visualize_final_path(animation_grid, final_path[::-1])
                save_waypoints(final_path[::-1], filename="final_path.txt")
                break
            else:
                print("\nGoal Point is on the obstacle")
        else:
            print("\nThe start point is on the obstacle")
            continue
    save_video(animation_frames)


if __name__ == "__main__":
    main()

/home/rahulk99/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


ModuleNotFoundError: No module named 'google.colab'

In [2]:
# -*- coding: utf-8 -*-
"""ENPM661_project3_with_smoothing.ipynb"""

from numpy import cos, sin, deg2rad, round, sqrt
import time
import heapq
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# --- PARAMETERS ---
canvas_height, canvas_width = 300, 540
wheel_radius = 3.6
wheel_distance = int(235 / 100) * 10  # mm
robot_radius = int(33.5 / 2)  # mm
goal_threshold = 1.5
visited_threshold = 1.0
RPM1_val, RPM2_val = 10, 20  # default values, will be updated in main()
current_pose = [0.0, 0.0, 0.0]  # x, y, theta

# --- NODE CLASS ---
class Node:
    def __init__(self, coords, cost, parent=None, heuristic=0):
        self.coords = coords  # (x, y, theta)
        self.x = coords[0]
        self.y = coords[1]
        self.orientation = coords[2]
        self.cost = cost
        self.parent = parent
        self.heuristic = heuristic

    def __lt__(self, other):
        return self.cost + self.heuristic < other.cost + other.heuristic

# --- OBSTACLE DEFINITIONS ---
def is_in_obstacle_rect1(x, y, x0=100, y0=200, width=10, height=200, thickness=3):
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= canvas_height:
        return True
    return False

def is_in_obstacle_rect2(x, y, x0=210, y0=300, width=10, height=200, thickness=3):
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= height:
        return True
    return False

def is_in_obstacle_rect3(x, y, x0=320, y0=300, width=10, height=100, thickness=3):
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= height:
        return True
    return False

def is_in_obstacle_rect4(x, y, x0=320, y0=100, width=10, height=100, thickness=3):
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= canvas_height:
        return True
    return False

def is_in_obstacle_rect5(x, y, x0=430, y0=200, width=10, height=200, thickness=3):
    if x0 <= x <= x0 + thickness and canvas_height - y0 <= y <= canvas_height:
        return True
    return False

# --- MAP GENERATION ---
def generate_map(clearance):
    grid = np.zeros((canvas_height, canvas_width), dtype=np.uint8)
    offset = clearance + robot_radius
    for i in range(canvas_height):
        for j in range(canvas_width):
            if (is_in_obstacle_rect1(j, i) or
                is_in_obstacle_rect2(j, i) or
                is_in_obstacle_rect3(j, i) or
                is_in_obstacle_rect4(j, i) or
                is_in_obstacle_rect5(j, i)):
                grid[i][j] = 1
    kernel = np.ones((int(offset * 2 + 1), int(offset * 2 + 1)), np.uint8)
    clearance_mask = cv2.dilate(grid, kernel, iterations=1)
    clearance_mask[np.where(grid == 1)] = 2
    return clearance_mask

def is_valid_point(x, y, grid):
    if grid[y][x] == 1:
        return False
    return True

def create_animation_grid(clearance):
    image = np.full((canvas_height, canvas_width, 3), 255, dtype=np.uint8)
    offset = clearance + robot_radius
    colors = {'clearance': (0, 255, 255), 'obstacle': (255, 0, 0)}
    obstacle_funcs = [is_in_obstacle_rect1, is_in_obstacle_rect2, is_in_obstacle_rect3, is_in_obstacle_rect4, is_in_obstacle_rect5]
    obstacle_mask = np.zeros((canvas_height, canvas_width), dtype=bool)
    for i in range(canvas_height):
        for j in range(canvas_width):
            obstacle_mask[i, j] = any(f(j, i) for f in obstacle_funcs)
    kernel = np.ones((offset*2 + 1, offset*2 + 1), np.uint8)
    clearance_mask = cv2.dilate(obstacle_mask.astype(np.uint8), kernel)
    image[clearance_mask.astype(bool) & ~obstacle_mask] = colors['clearance']
    image[obstacle_mask] = colors['obstacle']
    return image

def round_value(number):
    return round(number * 2) / 2

# --- DIFFERENTIAL DRIVE MOTION ---
def differential_drive_action(x, y, theta, UL, UR, dt=0.05, steps=20):
    t = 0
    theta = deg2rad(theta)
    Xn, Yn, Thetan = x, y, theta
    for _ in range(steps):
        t += dt
        ul_rad = (UL * 2 * np.pi) / 60
        ur_rad = (UR * 2 * np.pi) / 60
        dx = 0.5 * wheel_radius * (ul_rad + ur_rad) * np.cos(Thetan) * dt
        dy = 0.5 * wheel_radius * (ul_rad + ur_rad) * np.sin(Thetan) * dt
        dtheta = (wheel_radius / wheel_distance) * (ur_rad - ul_rad) * dt
        Xn += dx
        Yn += dy
        Thetan += dtheta
    return [Xn, Yn, np.rad2deg(Thetan)]

def generate_successors(node, canvas):
    x, y, theta = node.coords
    cost = node.cost
    successors = []
    rpm_pairs = [
        (0, RPM1_val), (RPM1_val, 0), (RPM1_val, RPM1_val),
        (0, RPM2_val), (RPM2_val, 0), (RPM2_val, RPM2_val),
        (RPM1_val, RPM2_val), (RPM2_val, RPM1_val)
    ]
    for ul, ur in rpm_pairs:
        new_coords = differential_drive_action(x, y, theta, ul, ur)
        xn, yn, thetan = new_coords
        if 0 <= xn < canvas_width and 0 <= yn < canvas_height:
            if canvas[int(round(yn))][int(round(xn))] == 0:
                rounded_coords = [round_value(xn), round_value(yn), thetan % 360]
                new_cost = cost + np.linalg.norm([xn - x, yn - y])
                successors.append([rounded_coords, new_cost])
    return successors

# --- A* ALGORITHM ---
def astar_search(start_node, goal_node, canvas):
    node_graph = {}
    open_list = {}
    closed_list = {}
    queue = []
    open_list[str([start_node.x, start_node.y])] = start_node
    heapq.heappush(queue, [start_node.cost, start_node])
    while queue:
        _, current_node = heapq.heappop(queue)
        node_graph[str([current_node.x, current_node.y])] = current_node
        if sqrt((current_node.x - goal_node.x) ** 2 + (current_node.y - goal_node.y) ** 2) < goal_threshold:
            goal_node.parent = current_node.parent
            goal_node.cost = current_node.cost
            print("✅ Goal Reached!")
            break
        if str([current_node.x, current_node.y]) in closed_list:
            continue
        closed_list[str([current_node.x, current_node.y])] = current_node
        del open_list[str([current_node.x, current_node.y])]
        child_list = generate_successors(current_node, canvas)
        for child in child_list:
            child_x, child_y, child_theta = child[0]
            child_cost = child[1]
            if str([child_x, child_y]) in closed_list:
                continue
            child_heuristic = sqrt((goal_node.x - child_x) ** 2 + (goal_node.y - child_y) ** 2)
            child_node = Node([child_x, child_y, child_theta], child_cost, current_node, child_heuristic)
            if str([child_x, child_y]) not in open_list:
                open_list[str([child_x, child_y])] = child_node
                heapq.heappush(queue, [(child_cost + child_heuristic), child_node])
    return node_graph

def backtrack_path(node_graph, goal_node):
    path = []
    path.append([int(goal_node.x), int(goal_node.y), int(goal_node.orientation)])
    parent = list(node_graph.items())[-1][1].parent
    while parent:
        path.append([int(parent.x), int(parent.y), int(parent.orientation)])
        parent = parent.parent
    return path

# --- PATH SMOOTHING ---
def is_path_collision_free(p1, p2, canvas):
    x1, y1 = p1
    x2, y2 = p2
    num_points = int(np.linalg.norm([x2 - x1, y2 - y1]) // 0.5)
    for i in range(num_points + 1):
        u = i / num_points
        x = int(round((1-u)*x1 + u*x2))
        y = int(round((1-u)*y1 + u*y2))
        if x < 0 or y < 0 or x >= canvas.shape[1] or y >= canvas.shape[0]:
            return False
        if canvas[y, x] != 0:
            return False
    return True

def smooth_path(path, canvas):
    smoothed = [path[0]]
    idx = 0
    while idx < len(path) - 1:
        next_idx = len(path) - 1
        while next_idx > idx + 1:
            if is_path_collision_free((path[idx][0], path[idx][1]),
                                      (path[next_idx][0], path[next_idx][1]), canvas):
                break
            next_idx -= 1
        smoothed.append(path[next_idx])
        idx = next_idx
    return smoothed

# --- MAIN ---
def main():
    global RPM1_val, RPM2_val
    start_x, start_y, start_theta = map(int, input("\nEnter start (x y theta): ").split())
    goal_x, goal_y = map(int, input("\nEnter goal (x y): ").split())
    RPM1_val, RPM2_val = map(int, input("\nEnter RPM1 and RPM2: ").split())
    user_clearance = int(input("\nEnter clearance: "))
    real_clearance = user_clearance + 50  # Force 5cm extra for real-world

    if(start_x > canvas_width or start_y > canvas_height or goal_x > canvas_width or goal_y > canvas_height):
        print("\nStart or goal out of bounds")
        return

    canvas = generate_map(real_clearance)

    if is_valid_point(start_x, start_y, canvas) and is_valid_point(goal_x, goal_y, canvas):
        start_node = Node([start_x, canvas_height - start_y - 1, start_theta], 0)
        goal_node = Node([goal_x, canvas_height - goal_y - 1, 0], 0)
        node_graph = astar_search(start_node, goal_node, canvas)
        final_path = backtrack_path(node_graph, goal_node)

        # --- SMOOTHING ---
        smoothed_path = smooth_path(final_path[::-1], canvas)
        save_path(smoothed_path)

        # --- Visualization ---
        animation_grid = create_animation_grid(real_clearance)
        visualize_final_path(animation_grid, smoothed_path)

    else:
        print("\nStart or goal point inside obstacle")

def save_path(path, filename="final_path.txt"):
    with open(filename, "w") as f:
        for point in path:
            f.write("{}, {}, {}\n".format(point[0], point[1], point[2]))
    print("✅ Path saved to", filename)

def visualize_final_path(image, path):
    vis = image.copy()
    if not path:
        return vis
    start_point = tuple(path[0])
    goal_point = tuple(path[-1])
    cv2.circle(vis, start_point[:2], 4, (0, 255, 0), -1)
    cv2.circle(vis, goal_point[:2], 4, (0, 0, 255), -1)
    for i in range(1, len(path)):
        pt1 = tuple(path[i - 1])
        pt2 = tuple(path[i])
        cv2.line(vis, pt1[:2], pt2[:2], (255, 0, 255), 2)
    vis_rgb = cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(8, 6))
    plt.imshow(vis_rgb)
    plt.title("Smoothed Final Path")
    plt.axis("off")
    plt.show()

if __name__ == "__main__":
    main()



Enter start (x y theta):  30 30 0

Enter goal (x y):  500 200

Enter RPM1 and RPM2:  15 10

Enter clearance:  1



Start or goal point inside obstacle
